# `CDC Process`

## Enable CDC on the Entire Database

In [3]:
EXEC sys.sp_cdc_enable_db

Commands completed successfully.

Total execution time: 00:00:00.827

## Each table needs to be updated or added to system CDC database

In [4]:
EXEC sys.sp_cdc_enable_table  -- enable CDC for a table. The Area table in this case
@source_schema = N'dbo', 
@source_name   = N'Area', 
@role_name     = NULL

Commands completed successfully.

Total execution time: 00:00:01.785

## Shows which databases have CDC enabled

- If CDC is enabled on the database. 1 = yes

In [ ]:
SELECT [name], database_id, is_cdc_enabled  
FROM sys.databases

## Shows the tables enabled for CDC

In [ ]:
SELECT [name], is_tracked_by_cdc   -- lists the tables which are enabled by CDC
FROM sys.tables 
order by name

## Scan of all the CDC Changes

In [42]:
SELECT * from sys.dm_cdc_log_scan_sessions where empty_scan_count <> 0 


(14 rows affected)

Total execution time: 00:00:00.317

session_id,start_time,end_time,duration,scan_phase,error_count,start_lsn,current_lsn,end_lsn,tran_count,last_commit_lsn,last_commit_time,log_record_count,schema_change_count,command_count,first_begin_cdc_lsn,last_commit_cdc_lsn,last_commit_cdc_time,latency,empty_scan_count,failed_sessions_count
1,2021-08-23 21:53:44.360,2021-08-23 21:53:44.377,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,1,0
3,2021-08-23 21:54:14.367,2021-08-23 22:48:09.000,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,216,0
220,2021-08-23 22:48:39.093,2021-08-23 22:52:09.753,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,15,0
236,2021-08-23 22:52:39.847,2021-08-23 23:13:13.557,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,83,0
321,2021-08-23 23:13:58.667,2021-08-23 23:53:20.640,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,158,0
480,2021-08-23 23:53:50.703,2021-08-24 03:34:30.130,1,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,881,0
1362,2021-08-24 03:35:00.227,2021-08-27 22:11:24.477,17,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,21682,0
23046,2021-08-27 22:12:09.617,2021-08-27 22:28:27.540,1,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,66,0
23113,2021-08-27 22:28:57.633,2021-08-27 22:29:27.727,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,3,0
23117,2021-08-27 22:29:57.820,2021-08-27 22:30:12.870,0,Done,0,00000000:00000000:0000 ,00000000:00000000:0000 ,00000000:00000000:0000 ,0,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,0,0,00000000:00000000:0000 ,00000000:00000000:0000 ,1900-01-01 00:00:00.000,0,2,0


## Show all of the Changes for a CDC Enabled Table

- This is the SQL the Boomi operation will need to run for each table

In [62]:
SELECT   -- shows the changes for a CDC table, in this case the Area able      
      [__$start_lsn],
      [__$operation]      
      ,[AreaNo]
      ,[PlantNo]
      ,[AreaName]     
  FROM [cdc].[dbo_Area_CT] 
-- from this result, the following is checked on the _$operation column of the returned data set
-- =========
-- 1 = delete
-- 2 = insert
-- 3 = update (captured column values are those before the update operation). 
-- 4 = update (captured column values are those after the update operation)
-- For example:
--  __$operation	    AreaNo	PlantNo	AreaName
--  3	                218	    40	    test area
--  4	                218	    40	    test areas
-- =========

(0 rows affected)

Total execution time: 00:00:00.317

__$start_lsn,__$operation,AreaNo,PlantNo,AreaName


## Show the current retention policy of the CDC database

In [49]:
select * from cdc.cdc_jobs

(2 rows affected)

Total execution time: 00:00:00.324

job_type,maxtrans,maxscans,continuous,pollinginterval,retention,threshold
capture,500,10,0,0,0,0
cleanup,0,0,0,0,5,5000


## Changes the retention time, in minutes, of the CDC database

In [48]:
EXECUTE sys.sp_cdc_change_job
   @job_type = N'cleanup',
   @retention = 5 -- the retention time in minutes
   

Commands completed successfully.

Total execution time: 00:00:00.313

## Get the TIme From LSN

In [ ]:
-- get the time from the LSN
SELECT sys.fn_cdc_map_lsn_to_time(0x00000A1E000042400007)  